## Introduction to NLP

We will cover *practical* NLP:

* how to process text
* how to do useful things with it

... we will not cover *theoretical* NLP:

* grammars, syntax, linguistic models
* (sorry Noam)

In [ ]:
# download an IMDB dataset needed for later (~80 MB)
!wget -nc https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf aclImdb_v1.tar.gz

# download the English spacy tokenizer model we will use later
# this model is small (~7 MB)
!python -m spacy download en_core_web_sm

# spacy word vectors (47.1 MB)
!python -m spacy download en_core_web_md

# !pip install seriate

import os
import itertools
import re

from pathlib import Path

import numpy as np
from tqdm.notebook import tqdm
from spacy.tokens import Token
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn import tree

# used to display trees
import pydotplus
from IPython.display import Image
from matplotlib import cm

%matplotlib inline

def create_graph(mdl, cmap=None, feature_names=None):
    # cmap is a colormap
    # e.g. cmap = matplotlib.cm.coolwarm( np.linspace(0.0, 1.0, 256, dtype=float) )
    tree_graph = tree.export_graphviz(
        mdl, out_file=None,
        feature_names=feature_names,
        filled=True, rounded=True
    )
    graph = pydotplus.graphviz.graph_from_dot_data(tree_graph)
    
    # get colormap
    if cmap:
        # remove transparency
        if cmap.shape[1]==4:
            cmap = cmap[:,0:2]
        
        nodes = graph.get_node_list()
        for node in nodes:
            if node.get_label():
                # get number of samples in group 1 and group 2
                num_samples = [int(ii) for ii in node.get_label().split('value = [')[1].split(']')[0].split(',')]

                # proportion that is class 2
                cm_value = float(num_samples[1]) / float(sum(num_samples))
                # convert to (R, G, B, alpha) tuple
                cm_value = cm.coolwarm(cm_value)
                cm_value = [int(np.ceil(255*x)) for x in cm_value]
                color = '#{:02x}{:02x}{:02x}'.format(cm_value[0], cm_value[1], cm_value[2])
                node.set_fillcolor(color)

    Image(graph.create_png())
    return graph

def train_pipe_and_predict(pipe, X_train, y_train, X_test):
    """Convenience function for training pipeline."""
    print('Building model.')
    pipe.fit(X_train, y_train)

    print('Getting test predictions.')

    # Predicting with a test dataset
    pred = pipe.predict(X_test)
    
    return pred

def print_results(y_test, pred):
    # Model Accuracy
    print(f"Accuracy: {metrics.accuracy_score(y_test, pred):3.1%}")
    print(f"Precision: {metrics.precision_score(y_test, pred):3.1%}")
    print(f"Recall: {metrics.recall_score(y_test, pred):3.1%}")

def introspect_text(pipe, text):
    print(text, end='\n\n')
    vectorizer = pipe.named_steps['vectorizer']
    freq = vectorizer.transform([text])[0].toarray()
    features = vectorizer.get_feature_names_out()
    
    # get the prediction
    pred = pipe.predict([text])
    if pred[0] == 0:
        print('Prediction: NEGATIVE.')
    else:
        print('Prediction: POSITIVE.')
    for i, feat in enumerate(features):
        print(f'{feat:10s} {freq[0][i]:3d}')
        

# Why NLP?

<img src="img/nlp_text_with_ann.png">

* Dense source of information

# Why is NLP hard?

<img src="img/newspaper_san_jose.jpg" width=800>

# Why is NLP hard?

<img src="img/crash_blossoms.jpg" width=800>

[Garden-path sentence (wiki)](https://en.wikipedia.org/wiki/Garden-path_sentence) / [Crash Blossoms](https://languagelog.ldc.upenn.edu/nll/?cat=118)

# Why is NLP hard?

* ambiguous
* complex
* context dependent
* requires prior knowledge
* requires a world model
* is used for interaction with people (amusing, insulting)

# Goal of today

(Practical) NLP attempts to represent language as data, then do something useful with it.

A classic source of data are reviews (movie reviews, places, etc).

* Text of the review -> natural language
* Star rating of the review -> easy to convert to a label

Let's start with some very short reviews, and our goal is to classify whether they are positive (1) or negative (0) in terms of *sentiment*, i.e. whether someone liked the movie or not.

In [ ]:
sentences = [
    'Good times were had by all.',
    'This was a good-looking place.',
    'A once in a lifetime experience.',
    'Horrible.',
    'One of the worst experiences of my life.',
    "Everything went terribly, I'll never return."
]
y = [1, 1, 1, 0, 0, 0]

sentences_test = [
    'This was fun!',
    'Abysmal, horrible service.'
]
y_test = [1, 0]

## Create a feature matrix

* We need to transform a sentence into a vector of features ($\textbf{x}$)
* < w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>M</sub> >   ➞   **x**<sub>i</sub>

## Create a feature matrix

* We need to transform a `sentence` into a matrix of features (`X`).
* We can represent a `sentence` as a collection of symbols *s*

\[s<sub>1</sub>, s<sub>2</sub>, ..., s<sub>M</sub>\]  ➞ \[x<sub>1</sub>, x<sub>2</sub>, ..., x<sub>D</sub>\] 

Note $D$ (length of $\textbf{x}$) != $M$ (number of symbols in the sentence).

## Create a feature matrix

One feature for every letter in the alphabet

s<sub>1</sub> | s<sub>2</sub> | s<sub>3</sub> | s<sub>4</sub> | | to | | x<sub>1</sub> | x<sub>2</sub>
--- | --- | --- | --- | --- | --- | --- | --- | ---
d | i | p |   | | | | 0 | 1
c | a | t |   | | | | 0 | 0
h | i |   |   | | | | 1 | 0
s | h | i | p | | | | 1 | 1


In [ ]:
alphabet='abcdefghijklmnopqrstuvwxyz'
splitter = lambda x: re.split('', x)
def prepare_data(sentences, splitter=splitter, alphabet=alphabet):
    X = []
    for sentence in sentences:
        sentence = splitter(sentence)
        x = [int(symbol in sentence) for symbol in alphabet]
        X.append(x)
    return X
    
X = prepare_data(sentences)

In [ ]:
# visualize the data with the sentence
print(alphabet)
for i, sentence_vector in enumerate(X):
    print(''.join([str(c) for c in sentence_vector]), end=' ')
    print(sentences[i])

## Train the model

In [ ]:
mdl = tree.DecisionTreeClassifier(max_depth=3)

# fit the model to the data - trying to predict y from X
mdl = mdl.fit(X, y)

## Try our model on new data

In [ ]:
X_test = prepare_data(sentences_test)
pred_test = mdl.predict(X_test)

match = [pred_test[i] == y_test[i] for i, sentence in enumerate(sentences_test)]
print(f'{sum(match) / len(sentences_test):3.1%} accuracy.')
print([(pred_test[i], sentence) for i, sentence in enumerate(sentences_test)])

That's... bad. What happened?

## Visualize the trained model

In [ ]:
graph = create_graph(mdl, feature_names=list(alphabet))
Image(graph.create_png())

In [ ]:
# print out the training set with the letter "a" highlighted in each sentence
for i, sentence in enumerate(sentences):
    sentence = ''.join([f"\033[4m\033[91m{c}\033[0m"  if c == 'a' else c for c in sentence])
    print(f'{sentence}: {y[i]}')

Our model has learned that the letter "a" is always present in positive sentences, and never in negative sentences.

Let's get our hands on some real data.

# IMDb

A large dataset from the Internet Movie Database (IMDb) was curated by researchers [1].

- 50,000 reviews of movies on IMDB
- 25,000 positive reviews (>=7), 25,000 negative reviews (<=4)
- Commonly used for sentiment classification; i.e. is this review positive (great movie!) or negative (awful movie!)

> [1] Maas A, Daly RE, Pham PT, Huang D, Ng AY, Potts C. Learning word vectors for sentiment analysis. In Proceedings of the 49th annual meeting of the association for computational linguistics: Human language technologies 2011 Jun (pp. 142-150).

In [ ]:
def load_data(path):
    # initialize a dictionary (key-value store) for our corpus
    corpus = {}
    
    # for each file in the given folder path
    files = os.listdir(path)
    for fn in tqdm(files):
        filename = path / fn
        # open the file and read the text
        with open(filename, 'r') as fp:
            text = ''.join(fp.readlines())
        
        # put the text into the dictionary keyed by the filename
        # we can later access text data by indexing this dictionary
        # e.g. corpus["10_9"]
        corpus[filename.stem] = text
    
    return corpus

print('Loading positive documents (training set).')
pos = load_data(Path('aclImdb/train/pos'))
print('Loading negative documents (training set).')
neg = load_data(Path('aclImdb/train/neg'))

print('Loading positive documents (test set).')
pos_test = load_data(Path('aclImdb/test/pos'))
print('Loading negative documents (test set).')
neg_test = load_data(Path('aclImdb/test/neg'))

def get_numpy_arrays(pos, neg):
    # create numpy arrays of the datasets
    # these are more natively usable by ML libraries
    X_pos = np.array(list(pos.values()))
    X_neg = np.array(list(neg.values()))
    X = np.hstack([X_pos, X_neg])
    n_pos, n_neg = X_pos.shape[0], X_neg.shape[0]
    y = np.concatenate([np.ones(n_pos), np.zeros(n_neg)])
    return X, y

# only use ~1000 examples for speed
X_train, y_train = get_numpy_arrays(
    {k: v for k, v in pos.items() if k.startswith('2')},
    {k: v for k, v in neg.items() if k.startswith('2')}
)
X_test, y_test = get_numpy_arrays(
    {k: v for k, v in pos_test.items() if k.startswith('2')},
    {k: v for k, v in neg_test.items() if k.startswith('2')}
)

## Example review  - [Stanley and Iris](https://www.imdb.com/title/tt0100680/?ref_=tt_urv)

<img src="img/fonda_deniro_Stanley_And_Iris_1990.png" width=800>


## Example review  - [Stanley and Iris](https://www.imdb.com/title/tt0100680/?ref_=tt_urv)

In [ ]:
review = pos['10_9']
print(review)

# Redo our character example

In [ ]:
vocabulary='abcdefghijklmnopqrstuvwxyz'
splitter = lambda x: re.split('', x)

# visualize the data with the sentence
X = prepare_data(X_train, splitter, vocabulary)

mdl = tree.DecisionTreeClassifier(max_depth=3)

# fit the model to the data - trying to predict y from X
mdl = mdl.fit(X, y_train)

X = prepare_data(X_test, splitter, vocabulary)
pred_test = mdl.predict(X)

match = [pred_test[i] == y_test[i] for i in range(len(pred_test))]
print(f'{sum(match) / len(pred_test):3.1%} accuracy.')

graph = create_graph(mdl, feature_names=vocabulary)
Image(graph.create_png())

# How can we improve our representation of text?

Some things to note:

* we are *only* using letters [a-z]
  * we are ignoring capital letters
  * we are not using any punctuation
* we are just using the presence of the letter

So what do to?

* ...

* use more letters (expand vocabulary)
* use a morpheme (improve vocabulary)
* count the letters (better capture statistics about the text)

## Redo our character example with sklearn

With the mock dataset, we manually created the feature matrices with characters as features (a, b, c, d, ...).

Now we'll use scikit-learn for modelling. Read more about the components of scikit-learn. 
Remember, we have one goal: **represent text as data**... (to do *useful* things).

[Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline) are an *extremely* useful way to do these steps. It is basically a convenient API to tell Python "do A, then B, then C"... and to do it reproducibly with different datasets (think train/test splits!).

<img src="img/nlp_pipeline.png" width=800>

## Build the model

We'll focus on changing the arguments to `Pipeline` - and the utility functions defined above will use the pipeline to train a model using the training data and apply it to the test data.

In [ ]:
pipe = Pipeline([
    ('vectorizer', CountVectorizer(analyzer='char', binary=True) ),
    ('classifier', tree.DecisionTreeClassifier(max_depth=3))
])

# (note: pipeline isupdated in-place!)
pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)
# the above function simply calls:
#   pipe.fit(X_train, y_train)
#   pred = pipe.predict(X_test)

# Now that we have predictions for the test set (`pred`)
# we can calculate the performance of our model on a held out set.
print_results(y_test, pred)

## Visualize the model

[You can display pipelines graphically with scikit-learn!](https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_pipeline_display.html#sphx-glr-auto-examples-miscellaneous-plot-pipeline-display-py)

In [ ]:
from sklearn import set_config
set_config(display="diagram")
pipe

## Visualize the model

We can also look at the tree model.

In [ ]:
# grab the vectorizer / classifier to make a figure of the model
vectorizer = pipe.named_steps['vectorizer']
classifier = pipe.named_steps['classifier']

graph = create_graph(classifier, feature_names=vectorizer.get_feature_names_out())
Image(graph.create_png())

Note that our `CountVectorizer` class has added symbols beyond the alphabet.

In [ ]:
print([c for c in sorted(list(vectorizer.vocabulary_.keys()))])

Note that the above is slightly better than what we had before as we are including characters which are not letters (e.g. the asterisk, money sign, etc).

## Compare approaches

* Binary character presence: 57.8%.

## Count the characters rather than present/absent

In our above pipeline, we specified `binary=True`. From the [scikit-learn docs on CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html):

> binary **bool**, default=False
>
>    If True, all non zero counts are set to 1. This is useful for discrete probabilistic models that model binary events rather than integer counts.

This creates what we had before - a feature matrix of 1s and 0s. As the name implies, `CountVectorizer` is able to return a matrix of counts for how often a character appears. We can see the impact of changing this below.

In [ ]:
pipe = Pipeline([('vectorizer', CountVectorizer(
                    analyzer='char',
                    binary=False)
                 ),
                 ('classifier', tree.DecisionTreeClassifier(max_depth=3))])

# pipeline will be updated in-place!
pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)

# Now that we have predictions for the test set (`pred`), we can calculate the performance of our model on a held out set.
print_results(y_test, pred)

## Compare approaches

* Character presence: **57.8%**.
* Character counts: 55.4%.

# Recap

* We tried to classify some dummy text using the presence of letters
  * It did not go well
* We are going to try using the presence of words
  * We hope it will go better!


# The main problem: we are not using a morpheme

We have made a choice to represent our data based on the character.
However, *English characters do not carry that much meaning*.

Compare the words "Amazing" and "Abysmal"

* Both start with a capital 'A'
* Both have two of the letter 'a'
* Both have seven letters

... but clearly the words do not share meaning.

In [ ]:
# can get bytes for each character directly in python:
print(" ".join("{:02x}".format(ord(c)) for c in 'Amazing'))

# Deciding on a morpheme

Term | Text | Example split
--- | --- | ---
Byte | Amazing | 41 6d 61 7a 69 6e 67
Character | Amazing | A m a z i n g
Lexeme | Amazing | Amaz
Word | Amazing | Amazing
Multiple words | Wow that was amazing | was amazing
Sentence | Wow that was amazing | Wow that was amazing

(we can of course continue to higher levels: paragraphs, documents, corpus, ...).

# Deciding on a morpheme

Term | Text | Example split | Reasoning
--- | --- | --- | ---
~Byte~ | Amazing | 41 6d 61 7a 69 6e 67 | yikes
~Character~ | Amazing | A m a z i n g | too low level
Lexeme | Amazing | Amaz | not sure how to do this (yet!)
Word | Amazing | Amazing | interesting!
Multiple words | Wow that was amazing | was amazing | also interesting!
~Sentence~ | Wow that was amazing | Wow that was amazing | too specific, will overfit

(we can of course continue to higher levels: paragraphs, documents, corpus, ...).

In [ ]:
# look at our review again
review = pos['10_9']
print(review, end='\n\n')
# get all the unique words in the sentences, splitting by spaces
print(" | ".join([word for word in review.split(' ')[:10]]))

The above list.. isn't great.

* multiple words are combined ("I'm" is "I am")
* punctuation included in words
* plural is included so "movie" and "movies" would be different words

For this reason, we do not usually use *words* as our morpheme. We use *tokens*.

# Tokenization

* In NLP, we don't work with "words", we work with "tokens"
* *Tokens* are groups of symbols with some semantic meaning
* Tokenization is the process of converting text into individual tokens

# Tokenizers

A few very popular tokenizers:

* [spaCy's English tokenizer](https://spacy.io/api/tokenizer#_title)
* [nltk's English tokenizer](https://www.nltk.org/api/nltk.tokenize.html#module-nltk.tokenize)
* [huggingface's English Bert tokenizer](https://huggingface.co/docs/transformers/v4.17.0/en/model_doc/bert#transformers.BertTokenizer)

In [ ]:
import spacy
# load an English spacy model we have already downloaded
nlp = spacy.load("en_core_web_sm")

doc = nlp(review)
print([token for token in doc])

* `'m` and `'s` are given their own token
* punctuation marks are individual tokens
* whitespaces are individual tokens

As you can see, tokens *do not have to be words*!

... how do we turn this into `X`?

## Creating the feature matrix

* Recall when we used the alphabet
* Each column was a letter
* Each row was our sentence (now review)
  * `X[i,j] == 1` -> letter is in the document
  * `X[i,j] == 0` -> letter is **not** in the document

* We are using tokens instead of letters
  * **Advantage**: we think words will be a better representation of text meaning (compared to letters)
  * **Disadvantage**: we now need to create an "alphabet" for words, i.e. the **vocabulary**

# Vocabulary

* The set of tokens we choose is our *vocabulary*.
* Most vocabularies consist of the most common tokens.
* This works because word usage follows the Pareto principle.

"You will know 90% of the words in a book after the first 20 pages, so the rest of it will be easier to read.."

<table><tr>
<td>    <img src="https://upload.wikimedia.org/wikipedia/en/0/03/Secret_Adversary_First_Edition_Cover_1922.jpg" width=250px> </td>
<td>    <img src="https://upload.wikimedia.org/wikipedia/commons/a/ab/JoyceUlysses2.jpg" width=250px> </td>
<td>    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/6f/Joyce_wake.jpg/369px-Joyce_wake.jpg" width=250px> </td>
</tr></table>

<table><tr>
<td>    
<img src="...%0A">
</td>

<td>
<img src="...%0AOQdR3ybSIgAAAABJRU5ErkJggg==%0A">
</td>

<td>
<img src="...%0A">
</td>

</tr></table>

Source: [Book coverage by Roman Kierzkowski](https://github.com/vocapouch/vocapouch-research/blob/master/books/Book%20Coverage.ipynb)

You will know 90% of the words in a book after...

* 40 pages (16%) - Secret Adversary (75,208 words total)
* 221 pages (25%) - Ulysses (261,202 words total)
* 387 pages (53%) - Finnegans Wake (218,793 words total)

**Design decision**: let's use the most frequent 10 words in the training set as our vocabulary.

In [ ]:
def spacy_tokenizer(text):
    return [token.text for token in nlp.tokenizer(text)]

In [ ]:
pipe = Pipeline([('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, max_features=10, binary=True)),
                 ('classifier', tree.DecisionTreeClassifier(max_depth=3))])

# pipeline will be updated in-place!
pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)

# Model Accuracy
print_results(y_test, pred)

We now have 50% accuracy - no better than chance - yikes! What happened?

Well, we did select only the top 10 tokens (`max_features=10`). What do you think the top 10 tokens are in the IMDb dataset?

In [ ]:
from collections import Counter

token_freq = Counter()

for corpus in [pos, neg]:
    for text in tqdm(corpus.values(), total=len(corpus)):
        # only call the tokenizer for speed purposes
        doc = nlp.tokenizer(text)
        
        for token in doc:
            # use 'orth', the integer version of the string
            # will convert back to string version later
            token_freq[token.orth] += 1

# convert back to string
token_freq = Counter({nlp.vocab.strings[k]: v for k, v in token_freq.items()})

## The most popular word in each state
<img src="https://pbs.twimg.com/media/D627MpgW4AAqTNW?format=jpg&name=small">

In [ ]:
token_freq.most_common(10)

In [ ]:
# look at the features for a single review
# each row is a feature, and the 0/1 value is whether the feature is present in the review
text = pos['10_9']
print(text, end='\n\n')
freq = pipe['vectorizer'].transform([text])[0].toarray()
features = pipe['vectorizer'].get_feature_names_out()
for i, feat in enumerate(features):
    print(f'{feat:10s} {freq[0][i]:3d}')

# Stop words

* The tokens "the", ",", etc have little meaning on their own
* We call these "stop words".
* **Stop word removal**: delete these words from the text.

In [ ]:
def spacy_tokenizer(text):
    return [token.text for token in nlp.tokenizer(text) if not token.is_punct and not token.is_stop]

Note: There is no universal list of stop words! Everyone has their own flavor, including spacy.

In [ ]:
# train the model with our updated spacy_tokenizer, which removes stop words
pipe = Pipeline([('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, max_features=10, binary=True)),
                 ('classifier', tree.DecisionTreeClassifier(max_depth=3))])

pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)

print_results(y_test, pred)

In [ ]:
graph = create_graph(pipe['classifier'], feature_names=pipe['vectorizer'].get_feature_names_out())
Image(graph.create_png())

In [ ]:
text = pos['10_9']
introspect_text(pipe, text)

## Compare approaches

* Character binary: 57.8%.
* Character counts: 55.4%.
* Token binary: 49.5%.
* Token stop word removal binary: **59.7%**.

## Should you always remove stop words?

<img src="https://github.com/fastai/course-nlp/raw/master/images/skomoroch.png">

## Should you always remove stop-words?

Well - obviously the answer is no. But why was it helpful above?

Answer: We had **already destroyed our document structure**.

* The act of counting words removes any semblence of word order
* Stop words frequently only have meaning when retaining text structure
* Most obvious if you think about punctuation as stop words

# Recap: we are trying to represent text as data

So far, we have:

* tried to use presence of letters (it was bad)
* learned about tokenization (seems useful!)
* learned that not all tokens are important (stop-words).. but sometimes they are important

We are trying to do *useful* things with the data, and so when representing text, we have two considerations:

* Increase the **efficiency** of our representation
* Increase the **capacity** of our representation

# Recap: we are trying to represent text as data

**Efficiency**

* don't make the model learn something twice
* e.g. combine "movie" and "movies" into a single feature

**Capacity**

* give the model more flexibility to capture nuance
* e.g. increase vocabulary size

## Model improvement: counts instead of binary (capacity)

Just like before, we can output token counts rather than 1s and 0s indicating word presence. Note the change below to `binary=False`.

In [ ]:
# train the model with our updated spacy_tokenizer, which removes stop words
pipe = Pipeline([('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, max_features=10, binary=False)),
                 ('classifier', tree.DecisionTreeClassifier(max_depth=3))])

pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)

print_results(y_test, pred)

In [ ]:
vectorizer = pipe.named_steps['vectorizer']
classifier = pipe.named_steps['classifier']

# blue = positive, orange = negative
graph = create_graph(classifier, feature_names=vectorizer.get_feature_names_out())
Image(graph.create_png())

We can see that the counts are being incorporated into the model:

* "good" node requires at least 2 mentions
* "like" node requires at least 2 mentions

In [ ]:
# look at an example with one "great" and two "bad" - bottom right node - these are always negative sentiment
multiple_bad = [x for x in X_train if x.count('great') > 0.5 and x.count('bad') > 2.5]
print(multiple_bad[0])

## Compare approaches

* Character binary: 57.8%.
* Character counts: 55.4%.
* Token binary: 49.5%.
* Token stop word removal binary: 59.7%.
* Token stop word removal count: **63.0%**.

# Increase the vocabulary size (capacity)

We were restricting our model to only 10 tokens - but this is a very small number. We can double it easily.

In [ ]:
# we omit binary=False, as it is the default
pipe = Pipeline([('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, max_features=20)),
                 ('classifier', tree.DecisionTreeClassifier(max_depth=3))])

pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)

print_results(y_test, pred)

## Compare approaches

* Character binary: 57.8%.
* Character counts: 55.4%.
* Token binary: 49.5%.
* Token stop word removal binary: 59.7%.
* Token stop word removal count: 63.0%.
* Token stop word removal count 20 tokens: **63.2%**.

In [ ]:
text = pos['10_9']
introspect_text(pipe, text)

Notice the "movie" and "movies" - we have (functionally) the same word which is appearing twice. This is unideal - we would like to combine these together as the information we want to learn from a dataset would be the same for both.

## Compare approaches

We have done one of two things so far:

* Increased **efficiency** of our model with respect to the data
  * e.g. remove data we think won't help (stop words)
* Increased **capacity** of our model
  * e.g. add more features (10 -> 20)

# Stemming

- Many words have similar meaning but distinct morphology
  * this was entertaining // i was entertained
- Removing the end of the word is effective in grouping like terms
  * movies -> movie
  * entertaining -> entertain
- This process is called "stemming"

# Stemming implementations

* [Porter Stemmer](https://tartarus.org/martin/PorterStemmer/): Most common, non-aggressive
* [Snowball Stemmer](https://tartarus.org/martin/PorterStemmer/): Improvement over Porter Stemmer

[Snowball Stemmer implementation is on GitHub](https://github.com/snowballstem/snowball/blob/master/python/snowballstemmer/basestemmer.py).

```
define Step_2 as (
    [substring] R1 among (
        'tional'  (<-'tion')
        'enci'    (<-'ence')
        'anci'    (<-'ance')
```

In [ ]:
# spacy does not have a stemmer! We'll need to use nltk.
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer(language='english')

def spacy_tokenizer(text):
    return [stemmer.stem(token.text) for token in nlp.tokenizer(text) if not token.is_punct and not token.is_stop]

In [ ]:
pipe = Pipeline([('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, max_features=20)),
                 ('classifier', tree.DecisionTreeClassifier(max_depth=3))])

pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)

print_results(y_test, pred)

## Compare approaches

* Character binary: 57.8%.
* Character counts: 55.4%.
* Token binary: 49.5%.
* Token stop word removal binary: 59.7%.
* Token stop word removal count: 63.0%.
* Token stop word removal count 20 features: 63.2%.
* Token, stop word removal, count, 20 features, stemmed: **65.8%**.
  * improved *efficiency* of learning by merging similar words through lemmatization

In [ ]:
text = pos['10_9']
introspect_text(pipe, text)

* No longer dictionary words
* words like character and characters are combined into charact
* words like play and playing are also combined

# Lemmatization

* With stemming, we applied an algorithm to simplify the text
* With lemmatization, we use *knowledge* - a lexicon
  * swing == swung

In [ ]:
text = pos['10_9']
doc = nlp(text)
for token in doc[:4]:
    print(f'{token.text:15s} -> {token.lemma_:15s}')

In [ ]:
def spacy_tokenizer(text):
    # note we call nlp(text) now
    # lemmatizing is slow compared to tokenizing
    # HOWEVER, if using spacy, we should really use spacy's nlp.pipe, which lets us batch/multithread
    return [
        token.lemma_ for token in nlp(text)
        if not token.is_punct and not token.is_stop
    ]

In [ ]:
pipe = Pipeline([('vectorizer', CountVectorizer(tokenizer=spacy_tokenizer, max_features=20)),
                 ('classifier', tree.DecisionTreeClassifier(max_depth=3))])

pred = train_pipe_and_predict(pipe, X_train, y_train, X_test)

print_results(y_test, pred)

## Compare approaches

* Character binary: 57.8%.
* Character counts: 55.4%.
* Token binary: 49.5%.
* Token stop word removal binary: 59.7%.
* Token stop word removal count: 63.0%.
* Token stop word removal count 20 features: 63.2%.
* Token, stop word removal, count, 20 features, stemmed: 65.8%.
* **Token, stop word removal, count, 20 features, lemmatized: 68.2%**.
  * improved *efficiency* of learning by merging similar words through lemmatization

# What else could we do to improve performance?

Here's what we've done so far:

* binary features -> counts (efficiency)
* characters -> words (efficiency)
* preprocessing: stop word removal (efficiency)
* preprocessing: stem/lemmatize (efficiency)
* 10 features -> 20 features (capacity)

# What else could we do to improve performance?

Move beyond counting words as our features!

Vectorization Method | Considerations
--- | ---
One-hot encoding (binary) | All words equidistant, so normalization extra important
Term frequency (counting) | Most frequent words not always most informative
Term frequency + dim. reduction (e.g. PCA) | Assumes only linear correlation is informative
Term Frequency Inverse Document Frequency (TF–IDF) | Moderately frequent terms may not be representative of document topics slow for large vocab, assumes words independently drive similarity
Distributed Representations | TBD!

Read more at [Applied Text Analysis, Chapter 4, Text Vectorization and Transformation Pipelines](https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html).

# What else could we do to improve performance?

* Before: characters -> words
* Now: words -> groups of words, i.e. n-grams
* "hidden gem"

<img src="img/hidden_gem.jpeg" width=600>

# What else could we do to improve performance?

... whatever we can think of!

Let's look at an example from a recent paper.

# Example from a paper - Named Entity Recognition

Classify the individual tokens in a text.

In [ ]:
text = "Alistair Johnson saw me last Monday the 24th."
targets = ['NAME', 'NAME', 'O', 'O', 'O', 'DATE', 'DATE', 'DATE']

for i, token in enumerate(text.split(' ')):
    print(f'{token:15s} -> f(x) = {targets[i]}')

One interesting NLP task is deidentification: label words in a document as names, dates, etc. This task is extremely important for healthcare applications, and research data usually needs to be deidentified before use.

<img src="img/deid_approach.jpg" width=400>

[1] Lee HJ, Wu Y, Zhang Y, Xu J, Xu H, Roberts K. A hybrid approach to automatic de-identification of psychiatric notes. Journal of biomedical informatics. 2017 Nov 1;75:S19-27.
https://doi.org/10.1016/j.jbi.2017.06.006

# What type of features can we build?

* **Bag-of-words**: unigrams, bigrams and trigrams of words within a window of [-2, 2].

Same as before, **except** we have switched from text classification to named entity recognition. That is, we are classifying *words* rather than the entire document, so we'll have a vector for each word, and there is a "window" around each word from which we'll extract data.

<img src="img/nlp_word_window.png" width=500>


In [ ]:
unigrams = ['Monday', 'Tuesday']
print(f'{"Entity":15s} -> [{", ".join(unigrams)}, ...]')
tokens = text.split(' ')
for i, token in enumerate(tokens):
    start, stop = max(i-2, 0), min(i+2+1, len(tokens))
    feature = [int(gram in tokens[start:stop]) for gram in unigrams]
        
    print(f'{token:15s} -> {", ".join(map(str, feature))}, ...')

If we look at the word "me" - it is two words before Monday, so the feature is True (1). Contrast this with "saw", which is too many words away. This is a very rudimentary way of encoding context into the feature for a word.

* **Bag-of-words**: unigrams, bigrams and trigrams of words within a window of [-2, 2].
* **Part-of-speech (POS) tags**: unigrams, bigrams and trigrams of POS tags within a window of [−2, 2]. The Stanford POS Tagger [42] was used for POS tagging.

In [ ]:
# spaCy has nifty functionality for understanding some of its outputs through spacy.explain
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
for token in doc:
    print(f'{token.text:15s}{token.tag_:5s}{spacy.explain(token.tag_)}')

In [ ]:
# we can also show a parse tree
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

The above is a dependency parse of the sentence:

* Notice how there are words which is a root: "saw"
* The `nsubj` of "saw" in "Johnson"
* "Johnson" is a proper noun (`PROPN`).

Our model is described online: https://spacy.io/models/en#en_core_web_sm

Though we can use `spacy.explain` to figure out what a tag means.

* **Bag-of-words**: unigrams, bigrams and trigrams of words within a window of [-2, 2].
* **Part-of-speech (POS) tags**: unigrams, bigrams and trigrams of POS tags within a window of [−2, 2]. The Stanford POS Tagger [42] was used for POS tagging.
* **Word shapes**: mapping any or consecutive uppercase character(s), lowercase character(s), digit(s) and other character(s) in current word to ’ A’, ’ a’, ’ #’ and ’ -’ respectively. For instance, the word shapes of “Hospital” are “Aaaaaaaa” and “Aa”.

In [ ]:
word_shapes = []
for token in tokens:
    shape = ''
    for letter in token:
        if letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
            shape += 'A'
        # note the bug in the below if statement :)
        elif letter in 'abcdefghijklmnopqrstuvwxtz':
            shape += 'a'
        elif letter in '0123456789':
            shape += '#'
        else:
            shape += '-'
    word_shapes.append(shape)
    print(f'{token:15s} {shape}')

Note the mistake - forgot a 'y' in one of the if statements.

There must be a simpler way!

# Regular expressions (regex)

* Regex: match patterns in text
    * Concise and powerful pattern matching language
    * Supported by many computer languages, including SQL
* Challenges
    * Brittle
    * Hard to write, can get complex to be correct
    * Hard to read


In [ ]:
import re

text_modified = str(text)
print(text_modified)

Replace capital letters with 'A'.

In [ ]:
text_modified = re.sub('[A-Z]', 'A', text_modified)
print(text_modified)

Replace lower case letters with 'a'.

In [ ]:
text_modified = re.sub('[a-z]', 'a', text_modified)
print(text_modified)

Replace numbers with '#'.

In [ ]:
text_modified = re.sub('[0-9]', '#', text_modified)
print(text_modified)

Replace the rest with '-'.

In [ ]:
text_modified = re.sub('[^Aa# ]', '-', text_modified)
print(text_modified)

# Features extracted in this *one* paper

* **Bag-of-words**: unigrams, bigrams and trigrams of words within a window of [-2, 2].
* **Part-of-speech (POS) tags**: unigrams, bigrams and trigrams of POS tags within a window of [−2, 2]. The Stanford POS Tagger [42] was used for POS tagging.
* **Word shapes**: mapping any or consecutive uppercase character(s), lowercase character(s), digit(s) and other character(s) in current word to ’ A’, ’ a’, ’ #’ and ’ -’ respectively. For instance, the word shapes of “Hospital” are “Aaaaaaaa” and “Aa”.
* **Combinations of words and POS tags**: combining current word with the unigrams, bigrams and trigrams of POS tags within a window of [−1, 1], i.e. w0p−1, w0p0, w0p1, w0p−1p0, w0p0p1, w0p−1p1, w0p−1p0p1, where w0, p−1, p0 and p1 denote current word, last, current and next POS tags respectively.
* **Sentence information**: number of words in current sentence, whether there is an end mark at the end of current sentence such as ’.’, ’?’ and ’!’, whether there is any bracket unmatched in current sentence.
* **Affixes**: prefixes and suffixes of length from 1 to 5.
* **Orthographical features**: whether the word is upper case, contains uppercase characters, contains punctuation marks, contains digits, etc.
* **Section information**: twenty-nine section headers (see the supplementary file) were collected manually such as “History of Present Illness”; we check which section current word belongs to.
* **General NER information**: the Stanford Named Entity Recognizer [43] was used to generate the NER tags of current word, include: person, date, organization, location, and number tags, etc.

# Features extracted in this *one* paper

Happily, many NLP libraries have these features built in!

For example, named entity recognition.

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

# Word embeddings

We have slowly improved the representation of our text

Initially we had characters.

In [ ]:
# visualize the data with the sentence
alphabet = 'abcdefghijklmnopqrstuvwxyz'
X = prepare_data(sentences, alphabet=alphabet)
print(alphabet)
for i, sentence_vector in enumerate(X):
    print(''.join([str(c) for c in sentence_vector]), end=' ')
    print(sentences[i])

# Word embeddings

We have slowly improved the representation of our text

Then we moved to tokens, mostly word tokens.

In [ ]:
# visualize the data with the sentence
alphabet = ['good', 'great', 'worst']
X = prepare_data(sentences, alphabet=alphabet)
print(' '.join([f'{c:10s}' for c in alphabet]))
for i, sentence_vector in enumerate(X):
    print(' '.join([f'{c:10d}' for c in sentence_vector]), end=' ')
    print(sentences[i])

# Word embeddings

Intuitively, we know "good" and "great" are similar.

**But the model has no mechanism for accounting for this**. Anything it learns about good, it must also learn about great.

-> we don't *share* knowledge between "good" and "great"

# Manually creating word embeddings

We could manually create these features - especially if we have a thesaurus! Synonyms for good (as a noun):

1. that which is morally right; righteousness.
    * "virtue", "righteousness", "virtuousness", "goodness", "morality", "ethicalness", "uprightness", "upstandingness", "integrity", "principle", "dignity", "rectitude", "rightness", "honesty", "truth", "truthfulness", "honor", "incorruptibility", "probity", "propriety", "worthiness", "worth", "merit", "irreproachableness", "blamelessness", "purity", "pureness", "lack of corruption", "justice", "justness", "fairness"
2. benefit or advantage to someone or something.
    * "benefit", "advantage", "profit", "gain", "interest", "welfare", "well-being", "enjoyment", "satisfaction", "comfort", "ease", "convenience", "help", "aid", 

# Manually creating word embeddings


But "good" has different meanings depending on its part of speech. When used as an adjective, we have many more options.. 

3. having the qualities required for a particular role.
    * "fine", "quality", "superior", "excellent", "superb", "outstanding", "magnificent", "exceptional", "marvelous", "wonderful", "first-rate", "first-class", "splendid", "ambrosial"
4. skilled at doing or dealing with a specified thing.
    * "capable", "able", "proficient", "adept", "adroit", "accomplished", "seasoned", "skillful", "skilled", "gifted", "talented", "masterly", "virtuoso", "expert", "knowledgeable", "qualified", "trained", "great", "mean", "wicked", "deadly", "nifty", "crack", "super", "ace", "wizard", "magic"
5. useful, advantageous, or beneficial in effect.
    * "wholesome", "health-giving", "healthful", "healthy", "nourishing", "nutritious", "nutritional", "strengthening", "beneficial", "salubrious", "salutary"
6. appropriate to a particular purpose.
    * "convenient", "suitable", "appropriate", "fitting", "fit", "suited", "agreeable", "opportune", "timely", "well timed", "favorable", "advantageous", "expedient", "felicitous", "propitious", "auspicious", "happy", "providential", "commodious", "seasonable"
7. possessing or displaying moral virtue.
    * "virtuous", "righteous", "moral", "morally correct", "ethical", "upright", "upstanding", "high-minded", "right-minded", "right-thinking", "principled", "exemplary", "clean", "law-abiding", "lawful", "irreproachable", "blameless", "guiltless", "unimpeachable", "just", "honest", "honorable", "unbribable", "incorruptible", "anticorruption", "scrupulous", "reputable", "decent", "respectable", "noble", "lofty", "elevated", "worthy", "trustworthy", "meritorious", "praiseworthy", "commendable", "admirable", "laudable", "pure", "pure as the driven snow", "whiter than white", "sinless", "saintly", "saintlike", "godly", "angelic", "squeaky clean"
8. showing kindness.
    * "kind", "kindly", "kindhearted", "good-hearted", "friendly", "obliging", "generous", "charitable", "magnanimous", "gracious", "sympathetic", "benevolent", "benign", "altruistic", "unselfish", "selfless"
9. obedient to rules or conventions.
    * "well behaved", "obedient", "dutiful", "well mannered", "well brought up", "polite", "civil", "courteous", "respectful", "deferential", "manageable", "compliant", "acquiescent", "tractable", "malleable", "right", "correct", "proper", "decorous", "seemly", "appropriate", "fitting", "apt", "suitable", "convenient", "expedient", "favorable", "auspicious", "propitious", "opportune", "felicitous", "timely", "well judged", "well timed", "meet", "seasonable"
10. giving pleasure; enjoyable or satisfying.
    * "enjoyable", "pleasant", "agreeable", "pleasing", "pleasurable", "delightful", "great", "nice", "lovely", "amusing", "diverting", "jolly", "merry", "lively", "festive", "cheerful", "convivial", "congenial", "sociable", "super", "fantastic", "fabulous", "fab", "terrific", "glorious", "grand", "magic", "out of this world", "cool", "brilliant", "brill", "smashing", "peachy", "neat", "ducky", "beaut", "bonzer", "capital", "wizard", "corking", "spiffing", "ripping", "top-hole", "topping", "champion", "beezer", "swell", "frabjous"
11. (of clothes) smart and suitable for formal wear.
    * "best", "finest", "newest", "nice", "nicest", "smart", "smartest", "special", "party", "Sunday", "formal", "dressy", "Opposite", "casual", "scruffy", 


# Manually creating word embeddings

And that's not all! The meaning of "good" depends on its *context*.

12. Very good
  * "admirable", "brilliant", "exceptional", "fantastic"
13. Be good
  * "manage", "comport oneself", "act with decorum", "be civil"


We have one word and 13 features already - and we haven't even been exhaustive. We need a new approach. Wouldn't it be great if we could instead *learn* these features, rather than manually engineer them?

# Distributional hypothesis

* Does language have a distributional structure? [1]

*structure*:

> A set of phonemes or a set of data is structured in respect to some feature, to the extent that we can form in terms of that feature some organized system of statements which describes the members of the set and their interrelations (at least up to some limit of complexity).

*distributional*:

> The distribution of an element will be understood as the sum of all its environments. An environment of an element A is an existing array of its co-occurrents, i.e. the other elements, each in a  particular position, with which A occurs to yield an utterance.

-> you're known by the company you keep

[1] Harris ZS. Distributional structure. Word. 1954 Aug 1;10(2-3):146-62.
[PDF](https://www.tandfonline.com/doi/pdf/10.1080/00437956.1954.11659520).

# word2vec

Learn these features, rather than manually create them.

1. Take a whole bunch of text
2. Try to predict a word's *context* form the word itself ("skip-grams")
3. Use the model's representation of words (tokens)

(target word, context word)

<img src="img/word2vec_skipgram.png" width=400>

[1] https://www.tensorflow.org/tutorials/text/word2vec

This works great in theory - except we're going to end up with 10,000s of outputs for 10,000s of inputs.

Instead of predicting the *word*, let's just predict yes/no "is this word a neighbour?".

<img src='img/skipgram-with-negative-sampling.png' width="800">

[1] https://jalammar.github.io/illustrated-word2vec/

We also need to create negative samples - and to do that, we pair our word ("make") with randomly generated words from our vocabulary. The result is fantastically useful - instead of 10,000 features (0/1 for each token), we now have an *embedding*, which is a denser representation of the words along dimensions related to their use in sentences.

Further reading on word vectors

* https://blog.acolyer.org/2016/04/21/the-amazing-power-of-word-vectors/

In [ ]:
nlp = spacy.load("en_core_web_sm")
text = 'good great capable wholesome convenient virtuous kind virtue'
tokens = nlp(text)

vectors = []

for token in tokens:
    print(f'{token.text:10s}{token.has_vector} {token.vector_norm} {[f"{t:1.2f}" for t in token.vector[:5]]} ...')
    vectors.append(token.vector)

vectors = np.vstack(vectors)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=[12, 8])
sns.heatmap(vectors, yticklabels=[token.text for token in tokens])
plt.show()

Remember that the embedding dimension (x-axis) has no intrinsic order - so we can re-order it to make it easier to see relations

In [ ]:
from scipy.spatial.distance import pdist, cdist
from seriate import seriate
reorder_dims = seriate(pdist(vectors.T, metric='cosine'))
plt.figure(figsize=[12, 8])
sns.heatmap(vectors[:, reorder_dims], yticklabels=[token.text for token in tokens])
plt.gca().axes.xaxis.set_ticklabels([])
plt.show()

In [ ]:
nlp = spacy.load("en_core_web_md")
X_train_vectors = []
X_test_vectors = []
# only enable the vectorize step to speed up processing
with nlp.select_pipes(enable="tok2vec"):
    for text in tqdm(X_train, total=X_train.shape[0]):
        doc = nlp(str(text))
        # doc.vector contains the average of all token vectors
        X_train_vectors.append(doc.vector)

    for text in tqdm(X_test, total=X_test.shape[0]):
        doc = nlp(str(text))
        X_test_vectors.append(doc.vector)

X_train_vectors = np.vstack(X_train_vectors)
X_test_vectors = np.vstack(X_test_vectors)

In [ ]:
pipe = Pipeline([('classifier', tree.DecisionTreeClassifier(max_depth=3))])

pred = train_pipe_and_predict(pipe, X_train_vectors, y_train, X_test_vectors)

print_results(y_test, pred)

In [ ]:
classifier = pipe.named_steps['classifier']

# blue = positive, orange = negative
graph = create_graph(classifier)
Image(graph.create_png())

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# get 3 principle components
pca = PCA(n_components=2)
pca_result = pca.fit_transform(X_train_vectors)

# TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(pca_result)


plt.figure(figsize=(16,10))
sns.scatterplot(
    x=tsne_results[:,0], y=tsne_results[:,1],
    hue=y_train,
    palette=sns.color_palette("bright", 2),
    legend="full",
    alpha=0.3
)
plt.show()

# You have all you need to do NLP!

We wanted to represent text as data.

* We typically convert the text into numbers
* Learned about *morphemes*, and how we use tokenizers to create tokens
* Learned how we can increase *efficiency* of our representation (lemmatize), or increase *capacity* of our representation (larger vocabulary)
* Learned about *word vectors*, which are a very efficient way of representing text, which is feasibly learned from readily available text data


Feel free to follow along the rest of this intro to learn a bit more about how BERT and recent neural models allow incorporating *context* in a text representation.


# We haven't handled context yet!

We've learned what the word "good" means, but the meaning does not change between "be good" and "very good". Easier to think about this for homonyms (specifically homographs):

* The bank robber got away.
* The river bank is slippery.

# Sesame street to the rescue - [Illustrated BERT by Jay Alammar](https://jalammar.github.io/illustrated-bert/)

<img src='https://jalammar.github.io/images/transformer-ber-ulmfit-elmo.png' width="500">



# Sesame street to the rescue - [Illustrated BERT by Jay Alammar](https://jalammar.github.io/illustrated-bert/)

<img src='https://jalammar.github.io/images/elmo-embedding-robin-williams.png' width="500">

# Sesame street to the rescue - [Illustrated BERT by Jay Alammar](https://jalammar.github.io/illustrated-bert/)

<img src='https://jalammar.github.io/images/elmo-word-embedding.png' width="500">

# Experimenting with BERT

One good way is to experiment with Google's tutorial:

https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb

# What kind of tasks do people work on?

* Text classification
* Named Entity Recognition
* Part-of-speech tagging
* Question answering
* Entity Linking
* Relation extraction
* Topic modeling
* Language modeling
* Translation - human translation is a 33 billion dollar industry

Benchmarks
* [GLUE](https://gluebenchmark.com/)
* [SuperGLUE](https://super.gluebenchmark.com/tasks/)

# What libraries do people use?

* [scikit-learn](https://scikit-learn.org/stable/index.html)
* [nltk](https://www.nltk.org/index.html)
* [spaCy](https://spacy.io/)
* [fastai](https://www.fast.ai/)
* [PyTorch](https://pytorch.org/) / [HuggingFace](https://huggingface.co/)

Utilities used here

* [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) - convert text into features
* [User Guide on text feature extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) - a good intro into using scikit-learn for NLP
* [Decision Trees](https://scikit-learn.org/stable/modules/tree.html) - decision trees are a simple ML model - [colab tutorial on decision trees and more complex tree based models (e.g. xgboost)](https://colab.research.google.com/github/alistairewj/tree-prediction-tutorial/blob/master/trees-classification.ipynb)

## [spaCy pipeline](https://spacy.io/usage/processing-pipelines)

<img src="img/spacy_pipeline.svg" width=700>


- Tokenize: split text into individual meaningful units
- Pre-process: remove stop words, remove punctuation
- Stemming: remove the suffix of words to merge similar words together
- Lemmatize: use a lexicon to merge similar words together
- Extract features: do whatever you think makes sense!
  - [spaCy linguistic features](https://spacy.io/usage/linguistic-features)

# Fun datasets to try

- [ICLR 2021 Abstracts with Scores](https://github.com/evanzd/ICLR2021-OpenReviewData)
- [IMDb dataset](https://ai.stanford.edu/~amaas/data/sentiment/)
- [Jeopardy! questions](http://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file/)
- [MIMIC - contains deidentified free-text clinical notes](https://mimic.mit.edu/)
- [List of other NLP datasets](https://github.com/niderhoff/nlp-datasets)

# Further reading

* http://web.stanford.edu/class/cs224n/
* https://github.com/fastai/course-nlp
* http://norvig.com/ngrams/
* [The shallowness of Google translate](https://www.theatlantic.com/technology/archive/2018/01/the-shallowness-of-google-translate/551570/) - we still have some way to go!

# Thanks!

# Appendix

## A silly approach

We can memorize the sentences, and output the known answer.

In [ ]:
# create a dictionary which maps each sentence to a known outcome
model = {sentences[i]: y[i] for i in range(len(sentences))}
print('Model')
for sentence, target in model.items():
    print(f'  {sentence}: {target}')


# see how often our model is correct
print('\nPerformance')
match = [model[sentence] == y[i] for i, sentence in enumerate(sentences)]
print(f'{sum(match) / len(sentences):3.1%} accuracy.')

# What if we try a new sentence?

In [ ]:
sentence = "this was the best time of my life"
target = 0
model[sentence]

In [ ]:

try:
    match = [model[sentence] == y[i] for i, sentence in enumerate(sentences_test)]
    print(f'{sum(match) / len(sentences_test):3.1%} accuracy.')
except KeyError as err:
    print(f'Sentence not found in model!')

Here is the traceback for the error we would have had without the try/except block:

```python
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
<ipython-input-5-1d2b9ac74b7f> in <module>
----> 1 match = [model[sentence] == y[i] for i, sentence in enumerate(sentences_test)]
      2 print(f'{sum(match) / len(sentences_test):3.1%} accuracy.')

<ipython-input-5-1d2b9ac74b7f> in <listcomp>(.0)
----> 1 match = [model[sentence] == y[i] for i, sentence in enumerate(sentences_test)]
      2 print(f'{sum(match) / len(sentences_test):3.1%} accuracy.')

KeyError: 'This was fun!'
```


# Yikes!

Our model failed at out of sample data. Sentences are too specific - learning that a specific sentence is positive or negative does not help us generalize to other sentences. Clearly we need a new approach!

What if we picked a more generic, broadly applicable symbol? Characters come to mind!

# Words as data

* Our "alphabet" becomes all the words we've seen
  * Before: `alphabet = ['a', 'b', 'c', 'd', ...]`
  * Now: `alphabet = ['good', 'bad', 'movie', ...]`
  
... *except*, calling this an alphabet is a misnomer. We'll call it a *vocabulary*.

In [ ]:
# get all the unique words in the sentences, splitting by spaces
vocabulary = sorted(list(set(
    [word for sentence in sentences
     for word in sentence.split(' ')]
)))

In [ ]:
print(vocabulary)

The above list.. isn't great.

* punctuation included in words
* "experience" and "experiences" separate
* "A" and "a" are different
* "good-looking" vs "good"

> Hrm, that's annoying.

# Trying it out

We can try the token based approach on the dummy dataset.
It doesn't work well because our vocabulary is missing the words in the test set... so our model doesn't generalize.

In [ ]:
import spacy
# load an English spacy model we have already downloaded
nlp = spacy.load("en_core_web_sm")

for sentence in sentences:
    doc = nlp(sentence)
    print(sentence, end=': ')
    print([token for token in doc])

In [ ]:
# Applying to the dataset
# get all the unique words in the sentences, splitting by spaces
vocabulary = sorted(list(set(
    [str(token) for sentence in sentences
     for token in nlp(sentence)]
)))

print(f'Vocabulary size: {len(vocabulary)}.')
splitter = lambda x: [str(token) for token in nlp(x)]

# visualize the data with the sentence
X = prepare_data(sentences, splitter, vocabulary)
for i, sentence_vector in enumerate(X):
    print(''.join([str(c) for c in sentence_vector]), end=' ')
    print(sentences[i])

In [ ]:
# it's easiest to visualize this if we just use a pandas dataframe
# that will show us the column headers
import pandas as pd
df = pd.DataFrame(X, columns=vocabulary)
# note the transpose so we can see all the columns
# by default pandas omits middle columns if there are a lot, though it's configurable
df.T

In [ ]:
mdl = tree.DecisionTreeClassifier(max_depth=3)

# fit the model to the data - trying to predict y from X
mdl = mdl.fit(X, y)

X_test = prepare_data(sentences_test, splitter, vocabulary)
pred_test = mdl.predict(X_test)

match = [pred_test[i] == y_test[i] for i, sentence in enumerate(sentences_test)]
print(f'{sum(match) / len(sentences_test):3.1%} accuracy.')
print([(pred_test[i], sentence) for i, sentence in enumerate(sentences_test)])

graph = create_graph(mdl, feature_names=vocabulary)
Image(graph.create_png())

# Trying it out

We can try other sentences to explore the tokenizer's quirks.

In [ ]:
sentence = "To be or not to be; that is the question. Or at least it was Hamlet's question."
doc = nlp(sentence)
print([token for token in doc])